# **Практика 5. Агрегирование данных. Сводные таблицы.**

[SmartLMS](https://edu.hse.ru/mod/quiz/view.php?id=1918958)

Мы анализируем датасет регистраций товарных знаков в странах Евразийского экономического союза.
- `trademark_id`: Уникальный идентификационный номер товарного знака в системе.
- `country_registration`: Страна, в которой подана заявка на регистрацию товарного знака.
- `mktu_class`: Класс Международной классификации товаров и услуг (МКТУ), к которому отнесен товарный знак.
- `industry`: Отрасль экономики, в которой работает компания-заявитель.
- `applicant_type`: Организационно-правовая форма заявителя.
- `trademark_type`: Вид обозначения, регистрируемого в качестве товарного знака.
- `representative_firm`: Патентный поверенный или юридическая фирма, представляющая интересы заявителя.
- `priority_claimed`: Указание на заявление конвенционного или иного приоритета при подаче заявки.
- `examination_duration_days`: Срок проведения экспертизы заявки в календарных днях.
- `mktu_classes_count`: Количество классов МКТУ, указанных в одной заявке на товарный знак.
- `company_age_years`: Возраст компании-заявителя на момент подачи заявки в годах.
- `previous_applications`: Количество ранее поданных данным заявителем заявок на товарные знаки.
- `trademark_value_k`: Оценочная рыночная стоимость товарного знака в тысячах долларов США.
- `examination_count`: Количество проведенных экспертиз по заявке (формальная, substantiv, дополнительные).
- `similarity_percent`: Процент сходства с ранее зарегистрированными товарными знаками по результатам экспертизы.
- `maintenance_fee_usd`: Годовая плата за поддержание товарного знака в силе в долларах США.
- `use_before_registration_months`: Срок использования обозначения в хозяйственной деятельности до подачи заявки на регистрацию в месяцах.
- `opposition_count`: Количество поданных возражений третьих лиц против регистрации товарного знака.

In [1]:
import pandas as pd
df = pd.read_csv('data/trademark.csv')
df.head(2)

,trademark_id,country_registration,mktu_class,industry,applicant_type,trademark_type,representative_firm,priority_claimed,examination_duration_days,mktu_classes_count,company_age_years,previous_applications,trademark_value_k,examination_count,similarity_percent,maintenance_fee_usd,use_before_registration_months,opposition_count
0,TM20200000,Италия,Класс 04,Сельское хозяйство,ООО,Световой,Не указано,True,58.0,4,0.7,5,8.89,3,24.4,870.73,45.1,0
1,TM20200001,Казахстан,Класс 22,Сельское хозяйство,Индивидуальный предприниматель,Иероглифы,Не указано,True,34.0,6,1.4,1,6.70,2,7.3,1165.41,51.8,0


## **Задание 1**
Рассчитайте среднюю продолжительность (`use_before_registration_months`) использования товарного знака до его регистрации для российских заявителей (**Россия**; `country_registration`). Ответ округлите до двух знаков.



In [3]:
df[df['country_registration'] == 'Россия']['use_before_registration_months'].mean().round(2)

59.74

---

## **Задание 2**
Посчитайте количество уникальных юридических фирм (`representative_firm`), которые представляют **ЗАО** (`applicant_type`) со стоимостью товарного знака (`trademark_value_k`) ниже средней.



In [4]:
mean_trade = df['trademark_value_k'].mean()

df_flt = df[
    (df['applicant_type']=='ЗАО') & (df['trademark_value_k'] < mean_trade)
]

df_flt['representative_firm'].nunique()

50

---

## **Задание 3**
Найти общее количество возражений (`opposition_count`) для товарных знаков с максимальным количеством экспертиз (`examination_count`).



In [ ]:
max_exam = df['examination_count'].max()

df[
    df['examination_count'] == max_exam
]['opposition_count'].count()

11

---

## **Задание 4**
Посчитайте для скольки товарных знаков, которые относятся к классу 02 (**Класс 02**; `mktu_class`) и классу 30 (**Класс 30**; `mktu_class`), известен процент сходства (`similarity_percent`).



In [8]:
df[
    df['mktu_class'].isin(['Класс 02', 'Класс 30'])
]['similarity_percent'].count()

376

---

## **Задание 5**
Рассчитайте размах возраста компаний (`company_age_years`), которые не заявляли приоритет при подачи заявки (**False**; `priority_claimed`). Ответ округлите до двух знаков.

>*ПОДСКАЗКА: Размах - это разница между наибольшим и наименьшим значением*

In [9]:
df_flt = df[df['priority_claimed']==False]
df_flt['company_age_years'].max() - df_flt['company_age_years'].min()

32.6

---

## **Задание 6**
Постройте сводную таблицу, где по строкам расположены виды товарных знаков (`trademark_type`), а в столбце — средняя годовая плата за поддержание (`maintenance_fee_usd`) для каждого вида. В ответе укажите, вид товарного знака (`trademark_type`) с максимальной средней годовой платой за поддержание.

In [ ]:
df_pivot = (
    df
    .pivot_table(index='trademark_type', values='maintenance_fee_usd', aggfunc='mean')
    .sort_values('maintenance_fee_usd', ascending=False)
)

df_pivot.head(3)

,maintenance_fee_usd
trademark_type,
Форма товара,3262.047346
Шрифт,2084.950177
Буквы,1944.530876


**Ответ**: `Форма товара`

In [17]:
# или так
(
    df.groupby('trademark_type')['maintenance_fee_usd'].mean()
    .sort_values(ascending=False)
    .head(3)
)

trademark_type
Форма товара    3262.047346
Шрифт           2084.950177
Буквы           1944.530876
Name: maintenance_fee_usd, dtype: float64

---

## **Задание 7**
Постройте сводную таблицу, где по строкам расположены страны регистрации (`country_registration`), а в столбце — общее количество предыдущих заявок (`previous_applications`) для каждой страны. В ответе укажите, для скольких стран общее количество предыдущих заявок находится в диапазоне от 100 до 250 включительно.



In [33]:
df_pivot = df.pivot_table(index='country_registration', values='previous_applications', aggfunc='sum')
df_pivot = df_pivot[(df_pivot['previous_applications'] >= 100) & (df_pivot['previous_applications'] <= 250)]
df_pivot.shape[0]

23

**Ответ**: `23`

In [ ]:
# или так
df_pivot = df.groupby('country_registration')['previous_applications'].sum()
df_pivot = df_pivot[(df_pivot >= 100) & (df_pivot <= 250)]
df_pivot.shape[0]

23

---

## **Задание 8**
Постройте сводную таблицу, где по строкам расположены типы заявителей (`applicant_type`), а в столбце — количество уникальных отраслей (`industry`), в которых представлен каждый тип. В ответе укажите, тип заявителя (`applicant_type`), представленный в 32 отраслях.



In [37]:
df_pivot = df.pivot_table(index='applicant_type', values='industry', aggfunc='nunique')
df_pivot[df_pivot['industry']==32]

,industry
applicant_type,
Культурное учреждение,32


**Ответ**: `Культурное учреждение`

In [38]:
# или так
df_pivot = df.groupby('applicant_type')['industry'].nunique()
df_pivot[df_pivot==32]

applicant_type
Культурное учреждение    32
Name: industry, dtype: int64

---

## **Задание 9**
Постройте сводную таблицу, где по строкам расположены страны регистрации (`country_registration`), а в столбцах — средний срок рассмотрения (`examination_duration_days`) и средняя годовая плата за поддержание (`maintenance_fee_usd`) для каждой страны. По сводной таблице определите, в какой стране (`country_registration`) соотношение платы к сроку рассмотроения (`maintenance_fee_usd`/`examination_duration_days`) наименьшее.



In [44]:
df_pivot = df.pivot_table(
    index='country_registration',
    values=['examination_duration_days', 'maintenance_fee_usd'],
    aggfunc='mean'
)
df_pivot['val'] = df_pivot['maintenance_fee_usd'] / df_pivot['examination_duration_days']
df_pivot.sort_values('val').head(3)

,examination_duration_days,maintenance_fee_usd,val
country_registration,,,
Узбекистан,70.379310,994.613621,14.132188
Великобритания,65.510638,934.191915,14.260156
Туркменистан,71.886364,1094.820586,15.229879


**Ответ**: `Узбекистан`

In [43]:
# или так
df_pivot = df.groupby('country_registration')[['examination_duration_days', 'maintenance_fee_usd']].mean()
df_pivot['val'] = df_pivot['maintenance_fee_usd'] / df_pivot['examination_duration_days']
df_pivot.sort_values('val').head(3)

,examination_duration_days,maintenance_fee_usd,val
country_registration,,,
Узбекистан,70.379310,994.613621,14.132188
Великобритания,65.510638,934.191915,14.260156
Туркменистан,71.886364,1094.820586,15.229879


---

## **Задание 10**
Постройте сводную таблицу, где по строкам расположены виды товарных знаков (`trademark_type`), а в столбцах — общее количество классов МКТУ (`mktu_classes_count`) и общее количество возражений (`opposition_count`) для каждого вида. По сводной таблице определите, среднее количество возражений на один класс МКТУ (`opposition_count`/`mktu_classes_count`). В ответ укажите наибольшее значение созданного признака. Ответ округлите до двух знаков.


In [48]:
df_pivot = df.pivot_table(
    index='trademark_type',
    values=['mktu_classes_count', 'opposition_count'],
    aggfunc='sum'
)

df_pivot['mean_opposition'] = df_pivot['opposition_count'] / df_pivot['mktu_classes_count']
df_pivot['mean_opposition'].max().round(2)

0.37

**Ответ**: `0.37`

In [49]:
# или так
df_pivot = df.groupby('trademark_type')[['mktu_classes_count', 'opposition_count']].sum()
df_pivot['mean_opposition'] = df_pivot['opposition_count'] / df_pivot['mktu_classes_count']
df_pivot['mean_opposition'].max().round(2)

0.37

---

## **Задание 11**
Постройте сводную таблицу, где по строкам юридические фирмы, представляющие интересы заявителя (`representative_firm`), а в столбцах — минимальный и максимальный процент сходства товарного знака (`similarity_percent`) для каждой фирмы. По сводной таблице определите, для скольких фирм, размах процента сходства товарного знака (`similarity_percent`) больше среднего размаха, по сводной таблице.



In [50]:
df_pivot = df.pivot_table(
    index='representative_firm',
    values='similarity_percent',
    aggfunc=['min', 'max']
)

df_pivot['range'] = df_pivot['max'] - df_pivot['min']
(df_pivot['range'] > df_pivot['range'].mean()).sum()

25

**Ответ**: `25`

In [51]:
# или так
df_pivot = df.groupby('representative_firm')['similarity_percent'].agg(['min', 'max'])
df_pivot['range'] = df_pivot['max'] - df_pivot['min']
(df_pivot['range'] > df_pivot['range'].mean()).sum()

25

---

## **Задание 12**
Постройте сводную таблицу, где по строкам расположены классы МКТУ (`mktu_class`), а в столбцах — количество товарных знаков (`trademark_id`) и общий срок проведения формальной экспертизы заявки (`examination_duration_days`) для каждого класса. Найдите класс МКТУ (`mktu_class`), к которому относится наибольшее количество товарных знаков (`trademark_id`). В ответ укажите общее количество дней экспертизы (`examination_duration_days`) по этому классу.



In [53]:
df_pivot = df.pivot_table(
    index='mktu_class',
    aggfunc={'trademark_id': 'count', 'examination_duration_days': 'sum'}
)

df_pivot.sort_values('trademark_id', ascending=False).head(3)

,examination_duration_days,trademark_id
mktu_class,,
Класс 01,28385.0,412
Класс 02,23211.0,339
Класс 03,21767.0,311


**Ответ**: `28385`

In [55]:
df_pivot = df.groupby('mktu_class').agg({'trademark_id': 'count', 'examination_duration_days': 'sum'})
df_pivot.sort_values('trademark_id', ascending=False).head(3)

,trademark_id,examination_duration_days
mktu_class,,
Класс 01,412,28385.0
Класс 02,339,23211.0
Класс 03,311,21767.0
